# Project 3 Sample Code



Since we don't expect you to learn an entirely new approach to predictive modeling for this project, we instead encourage you to work with the models you already are familiar with in Project 1. 

For this project the easiest approach will simply be to condense the time series data into new features (e.g., engineer a feature for last month's sales, rolling averages, etc.) which would then allow you to treat each row as it's own independent data point. 

You can then simply use this month's sales data as the label, drop it from your dataframe and run a regression on it. 

This is certainly not the only approach you can take, and we highly encourage you experimenting with alternatives. But if you're stuck, this will give you a framework for getting started.

In [1]:
# import libraries 
import numpy as np 
import pandas as pd  

Let's first import our data.

In [2]:
# import the data 
dataunits = pd.read_csv('data/BrandTotalUnits.csv')
datasales = pd.read_csv('data/BrandTotalSales.csv')
datasales.head(20)

,Months,Brand,Total Sales ($)
0,09/2018,10x Infused,"1,711.334232"
1,09/2018,1964 Supply Co.,"25,475.21594500000"
2,09/2018,3 Bros Grow,"120,153.644757"
3,09/2018,3 Leaf,"6,063.5297850000000"
4,09/2018,350 Fire,"631,510.0481550000"
5,09/2018,710 Labs,"2,065,970.9803990000"
6,09/2018,A&A Craft Inc,"5,094.305340000000"
7,09/2018,AA Packaging,"2,333.3399880000000"
8,09/2018,Absolute Xtracts,"5,747,227.563172000"
9,09/2018,Aces Extracts,"155,523.768684"


In [3]:
datasales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25279 entries, 0 to 25278
Data columns (total 3 columns):
Months             25279 non-null object
Brand              25279 non-null object
Total Sales ($)    25279 non-null object
dtypes: object(3)
memory usage: 592.6+ KB


So first issue is the data in its current form isn't really useful to us, so let's do some conversion of our data.

In [4]:
#First convert our months to datetime
dataunits['Months'] = pd.to_datetime(dataunits['Months'])
#Total units is too large currently to convert to a float
#need to trim it first then convert to float
dataunits['Total Units'] = dataunits['Total Units'].str[8:]
dataunits['Total Units'] = pd.to_numeric(dataunits['Total Units'])


dataunits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27686 entries, 0 to 27685
Data columns (total 4 columns):
Brands              27686 non-null object
Months              27686 non-null datetime64[ns]
Total Units         25282 non-null float64
vs. Prior Period    24935 non-null float64
dtypes: datetime64[ns](1), float64(2), object(1)
memory usage: 865.3+ KB


In [5]:
#First convert our months to datetime
datasales['Months'] = pd.to_datetime(datasales['Months'])
#Total units is too large currently to convert to a float
#need to trim it first then convert to float
datasales['Total Sales ($)'] = datasales['Total Sales ($)'].str[8:]
datasales['Total Sales ($)'] = pd.to_numeric(datasales['Total Sales ($)'])


datasales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25279 entries, 0 to 25278
Data columns (total 3 columns):
Months             25279 non-null datetime64[ns]
Brand              25279 non-null object
Total Sales ($)    25254 non-null float64
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 592.6+ KB


## TimeSeries Feature Engineering 

So there's a number of ways of approaching this but given the complexity of multiple brands with overlapping time intervals what seems to work easiest for me is breaking the dataset up by brand, engineering the features you want for each brand, and then reassembling the new dataframe. 

In [6]:
brands = dataunits["Brands"].unique()
brands

array(['#BlackSeries', '101 Cannabis Co.', '10x Infused', ..., 'Zlixir',
       'Zoma', 'Zuma Topicals'], dtype=object)

In [7]:
for brand in brands:
    ...
#once you've successfully completed your feature engineering for 
#a single brand you can try wrapping it in a for loop to engineer 
#all brand features

For now I'l attempt to construct some features on a single brand

In [8]:
units = dataunits[dataunits.Brands == '101 Cannabis Co.']


In [9]:
units

,Brands,Months,Total Units,vs. Prior Period
5,101 Cannabis Co.,2019-11-01,2.000000e+00,NaN
6,101 Cannabis Co.,2019-12-01,NaN,-1.000000
7,101 Cannabis Co.,2020-01-01,4.800000e+08,NaN
8,101 Cannabis Co.,2020-02-01,3.100000e+08,1.016883
9,101 Cannabis Co.,2020-03-01,2.800000e+08,0.354169
10,101 Cannabis Co.,2020-04-01,2.000000e+07,-0.244750
11,101 Cannabis Co.,2020-05-01,3.200000e+08,-0.130045
12,101 Cannabis Co.,2020-06-01,5.000000e+09,-0.312484
13,101 Cannabis Co.,2020-07-01,9.400000e+08,0.383829
14,101 Cannabis Co.,2020-08-01,4.112000e+10,0.727218


### Feature Engineering

We'll now create two features based on sales history. I'm going to take last month's sales, as well as a rolling average of sales for the last three months.

In [10]:
# creating new dataframe from consumption column
#data_historic = units[['Total Units']]
# inserting new column with yesterday's consumption values
units.loc[:,'Previous Month'] = units.loc[:,'Total Units'].shift(-1)
# inserting another column with difference between yesterday and day before yesterday's consumption values.

units.loc[:,'Rolling Average'] = (units.loc[:,'Total Units'].shift(-1) + units.loc[:,'Total Units'].shift(-2) + units.loc[:,'Total Units'].shift(-3))/3


units

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:376: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,Brands,Months,Total Units,vs. Prior Period,Previous Month,Rolling Average
5,101 Cannabis Co.,2019-11-01,2.000000e+00,NaN,NaN,NaN
6,101 Cannabis Co.,2019-12-01,NaN,-1.000000,4.800000e+08,3.566667e+08
7,101 Cannabis Co.,2020-01-01,4.800000e+08,NaN,3.100000e+08,2.033333e+08
8,101 Cannabis Co.,2020-02-01,3.100000e+08,1.016883,2.800000e+08,2.066667e+08
9,101 Cannabis Co.,2020-03-01,2.800000e+08,0.354169,2.000000e+07,1.780000e+09
10,101 Cannabis Co.,2020-04-01,2.000000e+07,-0.244750,3.200000e+08,2.086667e+09
11,101 Cannabis Co.,2020-05-01,3.200000e+08,-0.130045,5.000000e+09,1.568667e+10
12,101 Cannabis Co.,2020-06-01,5.000000e+09,-0.312484,9.400000e+08,1.631333e+10
13,101 Cannabis Co.,2020-07-01,9.400000e+08,0.383829,4.112000e+10,2.590000e+10
14,101 Cannabis Co.,2020-08-01,4.112000e+10,0.727218,6.880000e+09,1.219333e+10


### Merging Data 

Now that we have only one brand to work with at a time, it's relatively trivial to merge our datasets and pull features from the other datasets. You can use this example.

In [11]:
sales = datasales[datasales.Brand == '101 Cannabis Co.']

sales

,Months,Brand,Total Sales ($)
7908,2019-11-01,101 Cannabis Co.,3.210000e+02
9146,2020-01-01,101 Cannabis Co.,6.356700e+04
9765,2020-02-01,101 Cannabis Co.,6.100700e+10
10424,2020-03-01,101 Cannabis Co.,7.053300e+10
11110,2020-04-01,101 Cannabis Co.,5.769200e+10
11799,2020-05-01,101 Cannabis Co.,9.447200e+10
12530,2020-06-01,101 Cannabis Co.,5.777300e+04
13267,2020-07-01,101 Cannabis Co.,1.410800e+04
14024,2020-08-01,101 Cannabis Co.,7.228000e+10
14808,2020-09-01,101 Cannabis Co.,8.832200e+04


In [12]:
units = units.merge(sales, left_on='Months', right_on='Months')

In [13]:
units = units.drop(['Brand'], 1)

In [14]:
units.head()

,Brands,Months,Total Units,vs. Prior Period,Previous Month,Rolling Average,Total Sales ($)
0,101 Cannabis Co.,2019-11-01,2.0,NaN,NaN,NaN,3.210000e+02
1,101 Cannabis Co.,2020-01-01,480000000.0,NaN,310000000.0,2.033333e+08,6.356700e+04
2,101 Cannabis Co.,2020-02-01,310000000.0,1.016883,280000000.0,2.066667e+08,6.100700e+10
3,101 Cannabis Co.,2020-03-01,280000000.0,0.354169,20000000.0,1.780000e+09,7.053300e+10
4,101 Cannabis Co.,2020-04-01,20000000.0,-0.244750,320000000.0,2.086667e+09,5.769200e+10


So now I have a dataframe with merged features and engineered features. I now want to read in some brand specific features to augment my dataset. 

## Brand Features Engineering

Let's see what we have here!

In [15]:
branddetails = pd.read_csv('data/BrandDetails.csv')

In [16]:
branddetails = branddetails[branddetails.Brand == '101 Cannabis Co.']

branddetails.head()

,State,Channel,Category L1,Category L2,Category L3,Category L4,Category L5,Brand,Product Description,Total Sales ($),...,Total THC,Total CBD,Contains CBD,Pax Filter,Strain,Is Flavored,Mood Effect,Generic Vendor,Generic Items,$5 Price Increment
4,California,Licensed,Inhaleables,Concentrates,Dabbable Concentrates,Wax,NaN,101 Cannabis Co.,101 Cannabis Co. - Afghan Kush - Wax,145.39627,...,0,0,THC Only,NaN,Afghan Kush,NaN,Not Mood Specific,Non-Generic Vendors,Non-Generic Items,$35.00 to $39.99
5,California,Licensed,Inhaleables,Concentrates,Dabbable Concentrates,Wax,NaN,101 Cannabis Co.,101 Cannabis Co. - Skywalker OG - Wax,"3,261.12486",...,0,0,THC Only,NaN,Skywalker OG,NaN,Not Mood Specific,Non-Generic Vendors,Non-Generic Items,$30.00 to $34.99
6,California,Licensed,Inhaleables,Concentrates,Dabbable Concentrates,Wax,NaN,101 Cannabis Co.,101 Cannabis Co. - Skywalker OG - Wax,"2,062.231412",...,0,0,THC Only,NaN,Skywalker OG,NaN,Not Mood Specific,Non-Generic Vendors,Non-Generic Items,$20.00 to $24.99
7,California,Licensed,Inhaleables,Concentrates,Dabbable Concentrates,Wax,NaN,101 Cannabis Co.,101 Cannabis Co. - Indica Strain Blends - Wax,62.556665,...,0,0,THC Only,NaN,Indica Strain Blends,NaN,Not Mood Specific,Non-Generic Vendors,Non-Generic Items,$10.00 to $14.99
8,California,Licensed,Inhaleables,Pre-Rolled,Infused Pre-Rolled,NaN,NaN,101 Cannabis Co.,101 Cannabis Co. - Hybrid Strain Blends - Infu...,"1,309.279796",...,0,0,THC Only,NaN,NaN,NaN,Not Mood Specific,Non-Generic Vendors,Generic Items,$25.00 to $29.99


I have a theory that it's important to determine if a company offers inhaleable and edible products as part of their product inventory so I'm going to create binary categorical features.

In [17]:
value = 0
value1 = 0

if 'Inhaleables' in branddetails['Category L1'].values:
    value = 1
if 'Edibles' in branddetails['Category L1'].values:
    value1 = 1
 
units['Inhaleables'] = value
units['Edible'] = value1


units

,Brands,Months,Total Units,vs. Prior Period,Previous Month,Rolling Average,Total Sales ($),Inhaleables,Edible
0,101 Cannabis Co.,2019-11-01,2.000000e+00,NaN,NaN,NaN,3.210000e+02,1,0
1,101 Cannabis Co.,2020-01-01,4.800000e+08,NaN,3.100000e+08,2.033333e+08,6.356700e+04,1,0
2,101 Cannabis Co.,2020-02-01,3.100000e+08,1.016883,2.800000e+08,2.066667e+08,6.100700e+10,1,0
3,101 Cannabis Co.,2020-03-01,2.800000e+08,0.354169,2.000000e+07,1.780000e+09,7.053300e+10,1,0
4,101 Cannabis Co.,2020-04-01,2.000000e+07,-0.244750,3.200000e+08,2.086667e+09,5.769200e+10,1,0
5,101 Cannabis Co.,2020-05-01,3.200000e+08,-0.130045,5.000000e+09,1.568667e+10,9.447200e+10,1,0
6,101 Cannabis Co.,2020-06-01,5.000000e+09,-0.312484,9.400000e+08,1.631333e+10,5.777300e+04,1,0
7,101 Cannabis Co.,2020-07-01,9.400000e+08,0.383829,4.112000e+10,2.590000e+10,1.410800e+04,1,0
8,101 Cannabis Co.,2020-08-01,4.112000e+10,0.727218,6.880000e+09,1.219333e+10,7.228000e+10,1,0
9,101 Cannabis Co.,2020-09-01,6.880000e+09,0.383160,2.970000e+10,1.006667e+10,8.832200e+04,1,0


I also believe that a total count of the number of products the brand offers is also a useful feature to include. Fortunately that's easy enough to determine!

In [18]:
productcount = (branddetails.Brand == '101 Cannabis Co.').count()

productcount

77

In [19]:
units['ProdCount'] = productcount

units.head()

,Brands,Months,Total Units,vs. Prior Period,Previous Month,Rolling Average,Total Sales ($),Inhaleables,Edible,ProdCount
0,101 Cannabis Co.,2019-11-01,2.0,NaN,NaN,NaN,3.210000e+02,1,0,77
1,101 Cannabis Co.,2020-01-01,480000000.0,NaN,310000000.0,2.033333e+08,6.356700e+04,1,0,77
2,101 Cannabis Co.,2020-02-01,310000000.0,1.016883,280000000.0,2.066667e+08,6.100700e+10,1,0,77
3,101 Cannabis Co.,2020-03-01,280000000.0,0.354169,20000000.0,1.780000e+09,7.053300e+10,1,0,77
4,101 Cannabis Co.,2020-04-01,20000000.0,-0.244750,320000000.0,2.086667e+09,5.769200e+10,1,0,77


The result is starting to look like a pretty darn good dataframe! We now have merged and engineered timeseries features, along with brand-level features included in our dataframe. 

To complete this work the next steps will be to: 

1. finalize our feature selection plan
2. consolidate these steps into a concise for loop for all brands and then append them into a single dataframe
3. finalize an imputation strategy
4. You can then treat the dataset like a typical regression problem where 'TotalSales' or 'TotalUnits' can be the label you predict on
5. As always report your metrics! (and speaking of metrics, I found this handy-dandy helper functin that spits out a bunch of useful ones for you...)

In [20]:
import sklearn.metrics as metrics
def regression_results(y_true, y_pred):
    # Regression metrics
    explained_variance=metrics.explained_variance_score(y_true, y_pred)
    mean_absolute_error=metrics.mean_absolute_error(y_true, y_pred) 
    mse=metrics.mean_squared_error(y_true, y_pred) 
    mean_squared_log_error=metrics.mean_squared_log_error(y_true, y_pred)
    median_absolute_error=metrics.median_absolute_error(y_true, y_pred)
    r2=metrics.r2_score(y_true, y_pred)
    print('explained_variance: ', round(explained_variance,4))    
    print('mean_squared_log_error: ', round(mean_squared_log_error,4))
    print('r2: ', round(r2,4))
    print('MAE: ', round(mean_absolute_error,4))
    print('MSE: ', round(mse,4))
    print('RMSE: ', round(np.sqrt(mse),4))